# SpinToolkit

Demo code for Spin wave theory for complex spin orders I: linear spin wave, generalized linear spin wave, multi-magnon continuum and finite-temperature effects_

Authors: Lei Xu, Xiaojian Shi, Yangjie Jiao, and [Zhentao Wang](https://orcid.org/0000-0001-7442-2933)

In this demo, we compute the 1-magnon excitation within $k$-space of the triangular lattice spin-$1/2$ model, focusing on a fixed energy.

## Prerequisites

import the modules and set the parallels

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from joblib import Parallel, delayed
import SpinToolkit_py as sptk

In [ ]:
# check the system information and choose the number of threads accordingly
sptk.print_system_info()

n_jobs = 6 # set manually
# n_jobs = n_jobs   # use the maximum number of threads available

## Define lattice and hamiltonian

- triangular lattice with a 3-sublattice supercell compatible with 120$^\circ$ order

- XXZ model

- use "dipole" mode

In [ ]:
latt = sptk.lattice(filename = "triangular_K.toml")
Nsites = latt.Nsites()

In [ ]:
J = 1.0  # nearest-neighbor exchange
Δ = 0.9  # exchange anisotropy
h = 0.3  # Zeeman field

hamiltonian = sptk.model_spin(S = 0.5, mode = "dipole", lattice = latt)
print()

for site_i in range(Nsites):
        coor_i, sub_i = latt.site2coor(site = site_i)
        coor0_i, Mi   = latt.coor2supercell0(coor = coor_i)
        xi = coor_i[0]
        yi = coor_i[1]

        # 1st neighbor terms
        coor_j      = [xi + 1, yi]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, Δ * J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, Δ * J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        coor_j      = [xi + 1, yi + 1]
        coor0_j, Mj = latt.coor2supercell0(coor = coor_j)
        site_j      = latt.coor2site(coor = coor_j, sub = 0)
        hamiltonian.add_2spin_Jmatrix_XXZ(J = sptk.Vec3(J, J, Δ * J),
                                          site_i = site_i, site_j = site_j,
                                          Mi = Mi, Mj = Mj)

        # Zeeman field
        hamiltonian.add_zeeman(h = sptk.Vec3(0.0, 0.0, h), site = site_i)

hamiltonian.simplify()
print()
hamiltonian.build_mc_list()
print()

## Perform minimization (minimize from several independent initial random guesses, then pick the lowest-energy solution)

In [ ]:
print("Optimizing ground state", end = "... ", flush = True)
start        = time.perf_counter()
s_min, f_min = hamiltonian.optimize_spins_dipole(total_seeds = 20, seed0 = 0) # more seeds -> more likely to hit global minimum

end          = time.perf_counter()
print(f"{end - start}s")
print()

print(f"f_min (global): {f_min}")
for i in range(Nsites):
    print(f"site-{i}: {s_min[i]}")

## Initialize the LSW calculation and prepare the rotation matrices $\vec{s_{i}} = R_{i} {\hat z}$

In [ ]:
R = hamiltonian.init_LSW(s = s_min)
for i in range(Nsites):
    print(f"site-{i}:")
    print(R[i])
    print()

## Define 2D mesh in $k$-space, energy cut $\omega_0$ and Gaussian broadening factor $\sigma$ for computing $\mathcal{S}^{ab}({\bm k}, \omega)$

In [ ]:
# defines a 2D mesh in k-space by $\vec{k}_1 - \vec{k}_0$, $\vec{k}_2 - \vec{k}_0$

k0 = [-1.2, -1.6]
k1 = [ 1.2, -1.6]
k2 = [-1.2,  1.6]

km = sptk.k_map(dim=2, density = 600, k0 = k0, k1 = k1, k2 = k2)

ω0 = 0.6
ω_list = np.array([ω0])

# Gaussian broadening factor
FWHM  = 0.05
sigma = FWHM / 2.35482


## Compute 1-magnon DSSF $\mathcal{S}_{1}^{ab}({\bm k},\omega)$

In [ ]:
omega_k  = []
Sxx      = []
Syy      = []
Szz      = []
SxyPyx_R = []
SyzPzy_R = []
SzxPxz_R = []
SxyMyx_I = []
SyzMzy_I = []
SzxMxz_I = []


total_k = km.size()

def process_S1(index_k):
    k = km.k_list[index_k]
    if index_k % sptk.round2int(total_k / 50.0 + 1.0) == 0:
        print("*", end = "", flush = True)
    try:
        return sptk.DSSF_LSW(model = hamiltonian, R = R, T = 0.0,
                             k = k, omega_list = ω_list,
                             eval_1magnon = True, maxeval_2magnon = 0,
                             maxeval_3magnon = 0, maxeval_0magnon = 0,
                             broadening = "Gaussian", sigma_or_eta = sigma,
                             epsilon = 1.0e-5)
    except Exception as e:
        print(f"error: {e}, k = {k}")

start = time.perf_counter()
results_S1 = Parallel(n_jobs = n_jobs, prefer = "threads")(
    delayed(process_S1)(index_k) for index_k in range(total_k))
omega_k, Sxx, Syy, Szz, \
SxyPyx_R, SyzPzy_R, SzxPxz_R, \
SxyMyx_I, SyzMzy_I, SzxMxz_I = zip(*results_S1)
print()
end = time.perf_counter()
print(f"Used {(end - start) / 60.0:.6f}m")

In [ ]:

INSdatafile = f"INS_1magnon_energy{ω0}_triangular.txt"

# save and read data
num_k = len(np.unique(np.array(km.k_list)[:, 0]))
axis_X, axis_Y = np.meshgrid(np.unique(np.array(km.k_list)[:, 0]), np.unique(np.array(km.k_list)[:, 1]))
Intensity = (np.array(Sxx) + np.array(Syy) + np.array(Szz)).reshape(num_k, num_k)


# read data
# num_rows = num_k
# num_cols = num_k
# axis_X, axis_Y, Intensity = np.loadtxt(INSdatafile, unpack=True)
# axis_X = axis_X.reshape((num_rows, num_cols))
# axis_Y = axis_Y.reshape((num_rows, num_cols))
# Intensity = Intensity.reshape((num_rows, num_cols))



## Plot 1-magnon DSSF $\mathcal{S}_{1}({\bm k}, \omega) \equiv \mathcal{S}_{1}^{xx}({\bm k}, \omega) + \mathcal{S}_{1}^{yy}({\bm k}, \omega) + \mathcal{S}_{1}^{zz}({\bm k}, \omega)$

In [ ]:
plt.rcParams['figure.figsize'] = (4, 3)
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['font.size'] = 15

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath,newtxtext,newtxmath,bm}'
plt.rcParams['font.family'] = 'TeX Gyre Termes'

fig, ax = plt.subplots()

vmin = 0.0
vmax = 30
# convert
a0 = np.array([0.5 * np.sqrt(3), 0.5])
a1 = np.array([0.0, 1.0])

k1BZ = np.array([1/3, -1/3, -2/3,-1/3, 1/3, 2/3, 1/3])
k2BZ = np.array([1/3, 2/3, 1/3, -1/3, -2/3, -1/3, 1/3])
# other Bzs
ax.plot(k1BZ * a0[0] + k2BZ * a1[0] + a0[0],
		k1BZ * a0[1] + k2BZ * a1[1] + a0[1], '--', color='orange', linewidth = 0.8)

ax.plot(k1BZ * a0[0] + k2BZ * a1[0] - a1[0],
		k1BZ * a0[1] + k2BZ * a1[1] - a1[1], '--', color = 'orange', linewidth = 0.8)

ax.plot(k1BZ * a0[0] + k2BZ * a1[0] - a0[0] + a1[0],
		k1BZ * a0[1] + k2BZ * a1[1] - a0[1] + a1[1], '--', color='orange', linewidth = 0.8)

# 1BZ
ax.plot(k1BZ * a0[0] + k2BZ * a1[0], k1BZ * a0[1] + k2BZ * a1[1], '-', color='orange', linewidth = 0.8)

cmap = colors.LinearSegmentedColormap.from_list("white_to_blue", [(1, 1, 1), (0, 0, 1)], N = 256)

imag = ax.pcolormesh(axis_X[::2,::2] * a0[0] + axis_Y[::2,::2] * a1[0],
					 axis_X[::2,::2] * a0[1] + axis_Y[::2,::2] * a1[1],
					 Intensity[::2,::2], cmap = cmap, shading = "auto", vmin = 0, vmax = vmax)

xmin = -1.0
xmax = 1.0
ymin = -1.0
ymax = 1.0

ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_xticks([-1, 0, 1], [r"$-2\pi$", r"$0$", r"$2\pi$"])
ax.set_yticks([-1, 0, 1], [r"$-2\pi$", r"$0$", r"$2\pi$"])

ax.set_xlabel(r"$k_{x}$")
ax.set_ylabel(r"$k_{y}$")
ax.tick_params(direction = 'in', color = 'black')
ax.set_aspect('equal', adjustable='box')
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
	label.set_fontsize(15)

ax.text(0.1 * xmax, 0.75 * ymax,
		fr'$\omega / J = {ω0}$',
		fontsize = 15,
		bbox=dict(facecolor='white',
		alpha = 0.5))

plt.savefig(f"Fig_INS_1magnon_energy{ω0}_triangular.pdf", bbox_inches = 'tight', pad_inches = 0)
plt.show()